# 🧪 wandb 통합 테스트

이 노트북은 wandb 로깅 및 추적 기능을 테스트합니다:
- wandb 초기화 및 로그 기록
- 주요 파라미터 및 결과 시각화

In [1]:
# [1] 프로젝트 루트 디렉토리로 이동
import os                                                   # OS 모듈 임포트
import sys                                                  # 시스템 모듈 임포트
os.chdir("../../../")                                       # 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())                      # 현재 디렉토리 출력

현재 작업 디렉토리: /home/ieyeppo/AI_Lab/computer-vision-competition-1SEN


In [2]:
# [2] 폰트 설정 및 경고 억제

# 경고 메시지 억제용 모듈 임포트
#!sudo apt -get install -y fonts-nanum  # 나눔폰트 설치 (필요시 주석 해제) 설치 후 vscode 재시작 필요
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 적용 및 시각화 환경 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔고딕 폰트 경로 및 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 폰트 등록 및 설정
fe = fm.FontEntry(fname=font_path, name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# 글자 겹침 방지를 위한 레이아웃 설정
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.titlepad'] = 20

In [3]:
# [3] 라이브러리 및 유틸리티 임포트
# wandb, 로거, 설정 등 필요한 모듈 임포트
import torch  # 파이토치 임포트
import wandb  # wandb 라이브러리 임포트
from src.logging.wandb_logger import WandbLogger  # wandb 로거 임포트
from src.utils.common import load_yaml  # 설정 로드 함수 임포트
from src.logging.unit_test_logger import create_test_logger  # 단위 테스트 로거 생성 함수 임포트
test_logger = create_test_logger("wandb_integration")  # 단위 테스트 로거 초기화

📝 단위 테스트 시작: wandb_integration
📝 로그 디렉토리: notebooks/modular/unit_tests/wandb_integration/20250906_003155


In [4]:
# [4] GPU 자동 체크
# GPU 사용 가능 여부 확인
if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
    print(f'✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}')  # GPU 이름 출력
else:
    print('⚠️ GPU 사용 불가, CPU로 실행됩니다')  # CPU 사용 안내

✅ GPU 사용 가능: NVIDIA GeForce RTX 4090


In [5]:
# [5] wandb 로깅 테스트
# wandb 초기화 및 로그 기록 테스트
try:  # 예외 처리 시작
    cfg = load_yaml("configs/train_highperf.yaml")  # 설정 파일 로드
    
    # wandb 로거 초기화 (config에서 파라미터 추출)
    project_name = cfg.get('wandb', {}).get('project_name', 'document-classification-team')
    experiment_name = cfg['project']['run_name']
    
    wandb_logger = WandbLogger(
        project_name=project_name,
        experiment_name=experiment_name,
        config=cfg
    )  # wandb 로거 초기화
    
    # wandb 실행 초기화
    wandb_logger.init_run()
    
    # 메트릭 로깅 테스트
    test_metrics = {'test_metric': 0.95, 'accuracy': 0.87}
    wandb_logger.log_metrics(test_metrics)  # 메트릭 로깅
    
    # wandb 종료
    wandb_logger.finish()
    
    # 테스트 결과 저장
    result = {
        'wandb_status': 'success',
        'test_metrics': test_metrics,
        'project_name': project_name,
        'experiment_name': experiment_name
    }
    test_logger.save_test_result('wandb_integration_test', result)
    test_logger.finalize_test()
    print('✅ wandb 로깅 테스트 완료')  # 테스트 완료 메시지
except Exception as e:  # 예외 발생 시
    print(f'⚠️ wandb 로깅 테스트 실패: {e}')  # 에러 메시지 출력
    # 에러 발생 시에도 로거 정리
    try:
        test_logger.save_test_result('wandb_integration_test', {'status': 'failed', 'error': str(e)})
        test_logger.finalize_test()
    except:
        pass

WandB 로그인 상태: ieyeppo-job


wandb: Currently logged in as: ieyeppo-job (kimsunmin0227-hufs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


📋 실험명: v094-swin-highperf-0906-0032
🔗 WandB URL: https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/75m2n2mx


accuracy,▁
test_metric,▁
accuracy,0.87
test_metric,0.95


📝 테스트 결과 저장: wandb_integration_test
✅ 테스트 완료! 총 소요 시간: 25.49초
✅ 결과 요약: notebooks/modular/unit_tests/wandb_integration/20250906_003155/test_summary.json

🏁 단위 테스트 완료: wandb_integration
📁 결과 디렉토리: notebooks/modular/unit_tests/wandb_integration/20250906_003155
⏱️ 소요 시간: 25.49초
📊 테스트 섹션 수: 1
✅ wandb 로깅 테스트 완료


## 📊 WandB 통합 테스트 결과 요약

### ✅ 테스트 성공 항목
1. **WandB 로그인**: `ieyeppo-job` 계정으로 성공적 로그인
2. **실험 초기화**: `v094-swin-highperf-0906-0024` 실험 생성
3. **메트릭 로깅**: test_metric(0.95), accuracy(0.87) 기록 완료
4. **프로젝트 연동**: `document-classification-team` 프로젝트 연결

### 📈 성능 지표
- **실행 시간**: 약 1분 14초 (73.82초)
- **기록된 메트릭**: 2개 (test_metric, accuracy)
- **WandB 파일**: 5개 파일 동기화 완료
- **로그 저장 위치**: `./wandb/run-20250906_002500-mevvam1c/logs`

### 📁 생성된 파일 구조
```
wandb/
├── run-20250906_002500-mevvam1c/
│   ├── logs/
│   └── files/
└── latest-run -> run-20250906_002500-mevvam1c

notebooks/modular/unit_tests/wandb_integration/20250906_002349/
├── test_summary.json
└── test_results.json
```

### 🔗 WandB 대시보드 정보
- **실험 URL**: [v094-swin-highperf-0906-0024](https://wandb.ai/kimsunmin0227-hufs/document-classification-team/runs/mevvam1c)
- **프로젝트 URL**: [document-classification-team](https://wandb.ai/kimsunmin0227-hufs/document-classification-team)
- **동기화 상태**: ✅ 완료 (5개 파일)
- **계정**: kimsunmin0227-hufs 팀

In [6]:
# [6] 실제 생성된 파일 구조 확인
import os
import glob
from datetime import datetime

print("🔍 WandB 통합 테스트 결과 파일 확인\n")

# 1. WandB 실행 디렉토리 확인
wandb_runs = glob.glob("wandb/run-*")
print(f"📁 WandB 실행 디렉토리 ({len(wandb_runs)}개):")
for run_dir in sorted(wandb_runs)[-3:]:  # 최근 3개만 표시
    if os.path.exists(run_dir):
        # 실행 ID 추출
        run_id = os.path.basename(run_dir)
        mtime = datetime.fromtimestamp(os.path.getmtime(run_dir))
        
        # 하위 파일들 확인
        log_dir = os.path.join(run_dir, "logs")
        files_dir = os.path.join(run_dir, "files")
        
        print(f"   ├── {run_id} (생성: {mtime.strftime('%Y-%m-%d %H:%M:%S')})")
        
        if os.path.exists(log_dir):
            log_files = os.listdir(log_dir)
            print(f"   │   ├── logs/ ({len(log_files)}개 파일)")
            
        if os.path.exists(files_dir):
            config_files = os.listdir(files_dir)
            print(f"   │   └── files/ ({len(config_files)}개 파일)")

# 2. latest-run 심볼릭 링크 확인
latest_run = "wandb/latest-run"
if os.path.exists(latest_run):
    target = os.readlink(latest_run) if os.path.islink(latest_run) else "디렉토리"
    print(f"\n🔗 최신 실행 링크: latest-run -> {target}")

# 3. 테스트 결과 디렉토리 확인
test_dirs = glob.glob("notebooks/modular/unit_tests/wandb_integration/202509*")
print(f"\n📊 단위 테스트 결과 디렉토리 ({len(test_dirs)}개):")
for test_dir in sorted(test_dirs)[-3:]:  # 최근 3개만 표시
    mtime = datetime.fromtimestamp(os.path.getmtime(test_dir))
    files = os.listdir(test_dir)
    json_files = [f for f in files if f.endswith('.json')]
    
    print(f"   ├── {os.path.basename(test_dir)} (생성: {mtime.strftime('%Y-%m-%d %H:%M:%S')})")
    for json_file in json_files:
        file_path = os.path.join(test_dir, json_file)
        size = os.path.getsize(file_path) / 1024  # KB
        print(f"   │   └── {json_file} ({size:.1f} KB)")

# 4. 방금 실행된 테스트의 상세 정보
if wandb_runs:
    latest_wandb_run = max(wandb_runs, key=os.path.getmtime)
    mtime = datetime.fromtimestamp(os.path.getmtime(latest_wandb_run))
    print(f"\n🕒 최신 WandB 실행:")
    print(f"   📁 디렉토리: {os.path.basename(latest_wandb_run)}")
    print(f"   📅 생성 시간: {mtime.strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n✅ WandB 통합 테스트 완료!")
print(f"📅 테스트 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 WandB 로깅, 메트릭 기록, 프로젝트 연동이 정상적으로 작동함을 확인했습니다.")

🔍 WandB 통합 테스트 결과 파일 확인

📁 WandB 실행 디렉토리 (16개):
   ├── run-20250905_173017-se816eog (생성: 2025-09-05 17:30:17)
   │   ├── logs/ (3개 파일)
   │   └── files/ (5개 파일)
   ├── run-20250906_002500-mevvam1c (생성: 2025-09-06 00:25:01)
   │   ├── logs/ (3개 파일)
   │   └── files/ (5개 파일)
   ├── run-20250906_003218-75m2n2mx (생성: 2025-09-06 00:32:19)
   │   ├── logs/ (3개 파일)
   │   └── files/ (5개 파일)

🔗 최신 실행 링크: latest-run -> run-20250906_003218-75m2n2mx

📊 단위 테스트 결과 디렉토리 (2개):
   ├── 20250906_002349 (생성: 2025-09-06 00:25:03)
   │   └── test_summary.json (0.6 KB)
   ├── 20250906_003155 (생성: 2025-09-06 00:32:21)
   │   └── test_summary.json (0.6 KB)

🕒 최신 WandB 실행:
   📁 디렉토리: run-20250906_003218-75m2n2mx
   📅 생성 시간: 2025-09-06 00:32:19

✅ WandB 통합 테스트 완료!
📅 테스트 일시: 2025-09-06 00:32:31
🎯 WandB 로깅, 메트릭 기록, 프로젝트 연동이 정상적으로 작동함을 확인했습니다.


## 🛠️ WandB 통합 테스트 트러블슈팅 가이드

### 🚨 자주 발생하는 문제들

#### 1. **ImportError: cannot import name 'WandbLogger'**
```python
# ❌ 잘못된 임포트 (대소문자 혼동)
from src.logging.wandb_logger import WandBLogger

# ✅ 올바른 임포트
from src.logging.wandb_logger import WandbLogger
```

#### 2. **WandB 로그인 문제**
```bash
# 터미널에서 수동 로그인
wandb login

# API 키가 필요한 경우
wandb login --relogin
```

#### 3. **메서드 호출 오류**
```python
# ❌ 존재하지 않는 메서드
wandb_logger.log_config(cfg)
wandb_logger.log_metric('key', value)

# ✅ 실제 사용 가능한 메서드
wandb_logger.init_run()
wandb_logger.log_metrics({'key': value})
wandb_logger.finish()
```

#### 4. **Config 파일 경로 문제**
- 작업 디렉토리가 프로젝트 루트인지 확인
- `configs/train_highperf.yaml` 파일 존재 여부 확인

### 📋 필수 확인 사항
1. **WandB 계정**: 로그인 상태 확인
2. **프로젝트 권한**: `document-classification-team` 프로젝트 접근 권한
3. **네트워크 연결**: WandB 서버 연결 상태
4. **Config 파일**: YAML 형식 및 필수 필드 존재

### 🔧 수동 테스트 방법
```python
# 1. 기본 wandb 테스트
import wandb
wandb.init(project="test-project", name="manual-test")
wandb.log({"test": 1})
wandb.finish()

# 2. WandbLogger 클래스 테스트
from src.logging.wandb_logger import WandbLogger
logger = WandbLogger(project_name="test", experiment_name="manual")
logger.init_run()
logger.log_metrics({"manual_test": 0.9})
logger.finish()
```